In [1]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import zipfile
from tqdm import tqdm
import base64
import pathlib

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [ ]:
os.listdir('pliki_z_fabryki/weapons_tests/do-not-share/2024_01_08.txt')

['2024_01_08.txt',
 '2024_01_17.txt',
 '2024_01_27.txt',
 '2024_01_29.txt',
 '2024_02_01.txt',
 '2024_02_11.txt',
 '2024_02_15.txt',
 '2024_02_21.txt',
 '2024_03_02.txt',
 '2024_03_12.txt',
 '2024_03_15.txt',
 '2024_03_18.txt',
 '2024_03_19.txt',
 '2024_03_25.txt',
 '2024_03_29.txt',
 '2024_03_31.txt',
 '2024_04_18.txt',
 '2024_04_27.txt',
 '2024_05_08.txt',
 '2024_05_14.txt',
 '2024_05_31.txt',
 '2024_06_02.txt',
 '2024_07_05.txt']

In [11]:
with open('pliki_z_fabryki/weapons_tests/do-not-share/2024_01_08.txt', 'r') as f:
    text = f.read().replace('\n', '')

In [ ]:
def get_text_embedding(txt_path):
    with open(txt_path, 'r') as f:
        text = f.read().replace('\n', '')
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [23]:
os.path.basename('pliki_z_fabryki/weapons_tests/do-not-share/2024_01_08.txt')

'2024_01_08.txt'

In [25]:
# Tworzenie embeddingów i indeksowanie
index = []
for report in tqdm(os.listdir('pliki_z_fabryki/weapons_tests/do-not-share/')):
    path = os.path.join('pliki_z_fabryki/weapons_tests/do-not-share/', report)
    embedding = get_text_embedding(path)
    index.append({"id": report, "embedding": embedding, "metadata": report})

100%|██████████| 23/23 [00:16<00:00,  1.42it/s]


In [27]:
query = "W raporcie, z którego dnia znajduje się wzmianka o kradzieży prototypu broni?"

In [28]:
query_embedding = client.embeddings.create(
    input=query,
    model="text-embedding-3-small"
).data[0].embedding

In [31]:
import numpy as np
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarities = []
for item in index:
    similarity = cosine_similarity(
        [query_embedding], [item["embedding"]]
    )[0][0]
    similarities.append({"id": item["id"], "similarity": similarity, "metadata": item["metadata"]})

In [34]:
similarities = sorted(similarities, key=lambda x: x["similarity"], reverse=True)

In [38]:
similarities[0]

{'id': '2024_02_21.txt',
 'similarity': np.float64(0.5604677405414169),
 'metadata': '2024_02_21.txt'}

In [45]:
similarities[0]['id'].split('.')[0].replace('_','-')

'2024-02-21'

In [46]:
solution_response = {
   "task": "wektory",
   "apikey": os.getenv('AI_DEVS_3_API_KEY'),
   "answer": similarities[0]['id'].split('.')[0].replace('_','-')
}

In [47]:
response = requests.post('https://centrala.ag3nts.org/report', json=solution_response)

In [48]:
response

<Response [200]>

In [49]:
response.json()

{'code': 0, 'message': '{{FLG:ZLODZIEJ}}'}